# Vacuum Cleaner World Simulation with Different Types of Agents

### Step 0
Plotting

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from environment import Environment

class VisualizeAgents:
    def __init__(self, env, agent, steps=100):
        self.env = env
        self.agent = agent
        self.steps = steps

    def visualize_floor_before(self):
        plt.title(str(self.agent) + ' Floor Before')
        plt.imshow(self.env.floor, cmap='Blues', interpolation='nearest')
        plt.show()

    def visualize_agentPath_before(self):
        plt.title(str(self.agent) + ' Path Before')
        plt.imshow(self.env.agent_has_been, cmap='Blues', interpolation='nearest')
        plt.show()
    
    def visualize_floor_after(self):
        plt.title(str(self.agent) + ' Floor After')
        plt.imshow(self.env.floor, cmap='Blues', interpolation='nearest')
        plt.show()
    
    def visualize_agentPath_after(self):
        plt.title(str(self.agent) + ' Path After')
        plt.imshow(self.env.agent_has_been, cmap='Blues', interpolation='nearest')
        plt.show()

### Step 01
Implement a simulation environment.
Must haves : environment must be a 2D grid of size M X N.
Environment must be initialized with some dirt randomly on X% of grids.
One grid can have more dirt than another grid.
Must have a performance measure.

In [2]:
import numpy as np

class Environment:
    def __init__(self, width, height):
        # Initialize the environment with the specified width and height.
        # Sets up the floor grid and agent's path grid.
        self.width = width
        self.height = height
        self.floor = np.zeros((height, width))  # Initialize floor grid
        self.agent_has_been = np.zeros((height, width))  # Initialize agent's path grid

    def add_dirt(self, dirt_percentage):
        # Add dirt to the environment randomly based on the given percentage.
        total_cells = self.width * self.height
        num_dirt_cells = int((dirt_percentage / 100) * total_cells)
        dirt_positions = np.random.choice(total_cells, num_dirt_cells, replace=False)

        for pos in dirt_positions:
            row = pos // self.width
            col = pos % self.width
            self.floor[row][col] = 1

    def is_dirty(self, x, y):
        # Check if the specified tile (x, y) is dirty.
        return self.floor[y][x] == 1

    def get_bounds(self):
        # Get the bounds of the environment (width, height).
        return (self.width, self.height)

    def update_env(self, x, y):
        # Update the environment by removing dirt at the specified tile (x, y).
        if self.is_dirty(x, y):
            self.floor[y][x] = 0

    def update_agent_path(self, x, y):
        # Record the agent's path by marking the specified tile (x, y) as visited.
        self.agent_has_been[y][x] = 2

    def get_stats(self):
        # Get statistics about the environment, including total dirt
        total_dirt = np.sum(self.floor)
        return total_dirt

    def visualize(self):
        # Visualize the environment (floor) and the agent's path.
        print("Floor:")
        print(self.floor)
        print("Agent Path:")
        print(self.agent_has_been)

### Step 02
Implement a randomized agent.
Think of this as a malfunctioning vacuum cleaner.
Randomized agent will move up, down, left, right or suck irrespective of what it percepts.

In [3]:
import numpy as np
from environment import Environment
import matplotlib.pyplot as plt 
class RandomAgent:

    def __init__(self, startX, startY):
        self.positionCol = startX
        self.positionRow = startY
        self.path = [(startX, startY)]
        self.dirty_cells_cleaned = [0]
        


    def action(self, env):
        #update agent environment
        env.update_agent_path(self.positionCol, self.positionRow)
        self.path.append((self.positionCol, self.positionCol))

        act = self.randomAction()
        match act:
            case "up"   : self.positionRow -= 1
            case "down" : self.positionRow += 1
            case "right": self.positionCol += 1
            case "left" : self.positionCol -= 1

        # stop vacuum if outside the grid
        if((self.positionRow < 0) or (self.positionRow >= env.get_bounds()[1])):
            return -1
        
        if((self.positionCol < 0) or (self.positionCol >= env.get_bounds()[0])):
            return -1

        #check if dirty and update the floor environment
        if(env.is_dirty(self.positionCol, self.positionRow)):
            env.update_env(self.positionCol, self.positionRow)
            self.dirty_cells_cleaned.append(self.dirty_cells_cleaned[-1] + 1)

    def randomAction(self):
        randomAct = np.random.choice(["up", "down", "right", "left"])
        return randomAct


    def __str__(self):
        return 'Random Agent'

### Step 03
Implement a reflex agent.
A agent that selects actions on the basis of the current percept.
A reflex agent will move up, down, left, right or suck depeding on the current precept.

In [4]:
import random
from environment import Environment

class reflexAgent:
    def __init__(self, start_x, start_y):
        self.curr_x = start_x
        self.curr_y = start_y
        self.path = [(start_x, start_y)]
        self.dirty_cells_cleaned = [0]
    def update_environment(self, env):
        env.update_env(self.curr_x, self.curr_y)

    def update_agent_path(self, action):
        if action == "UP":
            self.curr_x -= 1
        elif action == "DOWN":
            self.curr_x += 1
        elif action == "LEFT":
            self.curr_y -= 1
        elif action == "RIGHT":
            self.curr_y += 1

    def action(self, env):
        act = self.__reflex_action(env)
        #update the env
        if act == "SUCK":
            self.update_environment(env)
            self.dirty_cells_cleaned.append(self.dirty_cells_cleaned[-1] + 1)
        else:
        #update agent path
            self.update_agent_path(act)
            self.path.append((self.curr_x, self.curr_y))
        
    def __reflex_action(self, env):
        #add functionality to return a reflex action
        R = env.get_bounds()[1]
        C = env.get_bounds()[0]
        env.update_agent_path(self.curr_x, self.curr_y)

        if env.is_dirty(self.curr_x, self.curr_y):
            return "SUCK"
        elif self.curr_x == 0 and self.curr_y == 0:
            return random.choice(["DOWN", "RIGHT"]) 
        elif self.curr_x == 0 and self.curr_y == C - 1:
            return random.choice(["DOWN", "LEFT"])
        elif self.curr_x == R - 1 and self.curr_y == 0:
            return random.choice(["RIGHT", "UP"])
        elif self.curr_x == R - 1 and self.curr_y == C - 1:
            return random.choice(["LEFT", "UP"])
        elif self.curr_x == 0: 
            return random.choice(["DOWN", "RIGHT", "LEFT"])
        elif self.curr_x == R - 1: 
            return random.choice(["RIGHT", "LEFT", "UP"])  
        elif self.curr_y == 0: 
            return random.choice(["DOWN", "RIGHT", "UP"])
        elif self.curr_y == C - 1: 
            return random.choice(["DOWN", "LEFT", "UP"])  
        else:
            return random.choice(["DOWN", "UP", "LEFT", "RIGHT"])
       
    def visualize_agent_movement(self, env):
        #add functionality to visualize environment 
        print(self.curr_x, self.curr_y)
    
    def __str__(self):
        return 'Reflex Agent'

### Step 04
Implement a model based reflex agent.
A agent that selects actions on the basis of the percept history.
A model based reflex agent will move up, down, left, right or suck depending on the precept history.

In [5]:
from environment import Environment
import random

class ModelBasedReflexAgent:
    def __init__(self, start_x, start_y):
        self.location = (start_x, start_y)  
        self.history = {}
        self.path = [(start_x, start_y)]  # Initialize path with starting location
        self.dirty_cells_cleaned = [0]
    def sense(self, env):
        # Sensing the current location's state and update the agent's model
        x, y = self.location
        percept = "Dirty" if env.is_dirty(x, y) else "Clean"
        self.history[self.location] = percept
    def act(self, env):
        # Deciding the action based on the percept history
        C, R= env.get_bounds()
        x, y = self.location
        if self.history.get(self.location) == "Dirty":
            return "Suck"
        elif (x, y + 1) in self.history and self.history[(x, y + 1)] == "Dirty":
            return "Down"
        elif (x, y - 1) in self.history and self.history[(x, y - 1)] == "Dirty":
            return "Up"
        elif (x + 1, y) in self.history and self.history[(x + 1, y)] == "Dirty":
            return "Right"
        elif (x - 1, y) in self.history and self.history[(x - 1, y)] == "Dirty":
            return "Left"
        else:
            # If no dirty percept nearby, moves randomly
            possible_moves = []
            if y > 0: possible_moves.append("Up")
            if y < R-1: possible_moves.append("Down")
            if x > 0: possible_moves.append("Left")
            if x < C-1: possible_moves.append("Right")
            return random.choice(possible_moves)
    def move(self, action, env):
        # Moving the agent according to the action
        C, R= env.get_bounds()
        x, y = self.location
        if action == "Suck":
            env.update_env(x, y)
            self.dirty_cells_cleaned.append(self.dirty_cells_cleaned[-1] + 1)
        elif action == "Down" and y < R-1:
            self.location = (x, y+1)
            env.update_agent_path(x, y+1)
        elif action == "Up" and y>0:
            self.location = (x, y-1)
            env.update_agent_path(x, y-1)
        elif action == "Right" and x<C-1:
            self.location = (x+1, y)
            env.update_agent_path(x+1, y)
        elif action == "Left" and x>0:
            self.location = (x-1, y)
            env.update_agent_path(x-1, y)


        # Update path with the new location after any move or action
        self.path.append(self.location)
    
    def __str__(self):
        return 'Model Based Reflex Agent'

### Step 05
Run simulations.
Report average performance of 100 runs, for all three types of agents on grid sizes (5X5), (10 X 10), (100 X 100).

In [6]:
def main(rows, columns, dirt_percentage):
    """
    The main function to run the simulation of vacuum cleaner agents.
    """
    # Call the functions or instantiate the classes from each Python file



#---------------Random Agent average of 100 runs-----------#
    afterRandomAgent = []
    for ii in range(100):
        env=Environment(rows, columns)
        env.add_dirt(dirt_percentage)
        agent=RandomAgent(2, 2) # Creating an instance of the RandomAgent class

        dirty_tiles = env.get_stats()

        for i in range(100):
            res=agent.action(env)
            if (res==-1):
                break

        afterRandomAgent.append(dirty_tiles - env.get_stats())

    print('Average number of Tiles cleaned from Random Agent is: ' + str(sum(afterRandomAgent)/len(afterRandomAgent)))

    #plotting one run
    env1 = Environment(rows, columns)
    env1.add_dirt(dirt_percentage)
    agent1=RandomAgent(2, 2)
    visualizer1 = VisualizeAgents(env1, agent1)

    visualizer1.visualize_floor_before()
    visualizer1.visualize_agentPath_before()

    for i in range(100):
        res=agent1.action(env1)
        if (res==-1):
            break

    visualizer1.visualize_floor_after()
    visualizer1.visualize_agentPath_after()


#----------------Reflex Agent average of 100 runs--------------#

    afterReflexAgent = []
    for ii in range(100):
        env = Environment(rows, columns)
        env.add_dirt(dirt_percentage)

        agent=reflexAgent(2, 2) # Creating an instance of the RandomAgent class

        dirty_tiles = env.get_stats()

        for i in range(100):
            res=agent.action(env)

        afterReflexAgent.append(dirty_tiles - env.get_stats())

    print('Average number of Tiles cleaned from Reflex Agent is: ' + str(sum(afterReflexAgent)/len(afterReflexAgent)))

    #plotting one run
    env2 = Environment(rows, columns)
    env2.add_dirt(dirt_percentage)
    agent2=reflexAgent(2, 2)
    visualizer2 = VisualizeAgents(env2, agent2)

    visualizer2.visualize_floor_before()
    visualizer2.visualize_agentPath_before()

    for i in range(100):
        res=agent2.action(env2)

    visualizer2.visualize_floor_after()
    visualizer2.visualize_agentPath_after()


#-------------Model Based Reflex Agent average of 100 runs-----------#

    afterModelReflexAgent = []
    for ii in range(100):
        env = Environment(rows, columns)
        env.add_dirt(dirt_percentage)

        agent=ModelBasedReflexAgent(2, 2) # Creating an instance of the RandomAgent class

        dirty_tiles = env.get_stats()

        for i in range(100):
            agent.sense(env)
            action=agent.act(env)
            agent.move(action, env)

        afterModelReflexAgent.append(dirty_tiles - env.get_stats())

    print('Average number of Tiles cleaned from Model Reflex Agent is: ' + str(sum(afterModelReflexAgent)/len(afterModelReflexAgent)))

    #plotting one run
    env3 = Environment(rows, columns)
    env3.add_dirt(dirt_percentage)
    agent3=ModelBasedReflexAgent(2, 2)
    visualizer3 = VisualizeAgents(env3, agent3)

    visualizer3.visualize_floor_before()
    visualizer3.visualize_agentPath_before()

    for i in range(100):
        agent3.sense(env3)
        action=agent3.act(env3)
        agent3.move(action, env3)

    visualizer3.visualize_floor_after()
    visualizer3.visualize_agentPath_after()

In [7]:
main(10, 10, 40)

AttributeError: 'int' object has no attribute 'update'